In [1]:
!pip install -q tensorflow-text


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import tensorflow as tf
import tensorflow_text as tf_text
import pandas as pd
import numpy as np
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import re
import string

E0000 00:00:1731179705.296869      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1109 19:15:05.305261203      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1109 19:15:05.305275323      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1109 19:15:05.305278625      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1109 19:15:05.305281064      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1109 19:15:05.305283368      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [3]:
# Load data
train_data = pd.read_csv('/kaggle/input/hackathon/train.csv')
test_data = pd.read_csv('/kaggle/input/hackathon/test.csv')

# Combine category and sub_category into a single label
train_data['label'] = train_data['category'] + '-' + train_data['sub_category']
test_data['label'] = test_data['category'] + '-' + test_data['sub_category']

# Ensure all labels are treated as strings
train_data['label'] = train_data['label'].astype(str)
test_data['label'] = test_data['label'].astype(str)

# Add a placeholder for unknown labels in training data
placeholder = {'label': 'Unknown-Unknown'}
# Use pd.concat to add the placeholder row
train_data = pd.concat([train_data, pd.DataFrame([placeholder])], ignore_index=True)

# Define a function to clean text
def clean_text(text):
    text = str(text)  # Convert to string explicitly
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Clean the text data
train_data['crimeaditionalinfo'] = train_data['crimeaditionalinfo'].apply(clean_text)
test_data['crimeaditionalinfo'] = test_data['crimeaditionalinfo'].apply(clean_text)

# Encode labels, handling unseen labels by mapping them to the placeholder
le = LabelEncoder()
le.fit(train_data['label'])
train_data['label_encoded'] = le.transform(train_data['label'])
test_data['label_encoded'] = test_data['label'].apply(lambda x: le.transform([x])[0] if x in le.classes_ else le.transform([placeholder['label']])[0])

# Split the data
X_train, X_val, y_train, y_val = train_test_split(train_data['crimeaditionalinfo'], train_data['label_encoded'], test_size=0.2, random_state=42)



In [4]:
max_tokens = 20000
max_len = 300

vectorize_layer = TextVectorization(max_tokens=max_tokens, output_sequence_length=max_len)
vectorize_layer.adapt(X_train.values)

model = Sequential([
    vectorize_layer,
    Embedding(max_tokens, 128, mask_zero=True),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


I0000 00:00:1731179733.263562      13 service.cc:145] XLA service 0x5675c7220f10 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731179733.263619      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1731179733.263624      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1731179733.263627      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1731179733.263629      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1731179733.263632      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1731179733.263635      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1731179733.263637      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1731179733.263641      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 254s 214ms/step - accuracy: 0.3513 - loss: 2.1935 - val_accuracy: 0.5000 - val_loss: 1.6185
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 251s 214ms/step - accuracy: 0.5245 - loss: 1.5304 - val_accuracy: 0.5376 - val_loss: 1.4811
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 250s 213ms/step - accuracy: 0.5795 - loss: 1.3371 - val_accuracy: 0.5441 - val_loss: 1.4560
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 251s 214ms/step - accuracy: 0.6216 - loss: 1.2039 - val_accuracy: 0.5406 - val_loss: 1.5035
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 250s 214ms/step - accuracy: 0.6666 - loss: 1.0702 - val_accuracy: 0.5324 - val_loss: 1.5582
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 252s 215ms/step - accuracy: 0.7031 - loss: 0.9506 - val_accuracy: 0.5217 - val_loss: 1.6703
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 249s 212ms/step - accuracy: 0.7384 - loss: 0.8343 - val_accuracy: 0.5114 - val_loss: 1.7994
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 252s 215ms/step - ac

In [8]:
# Step 5: Model Evaluation
y_pred = np.argmax(model.predict(X_val), axis=-1)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, average='weighted', zero_division=0))
print("Recall:", recall_score(y_val, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_val, y_pred, average='weighted'))


586/586 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step
Accuracy: 0.4689934891663998
Precision: 0.45696493042484243
Recall: 0.4689934891663998
F1 Score: 0.4593902167832193


In [7]:
test_predictions = np.argmax(model.predict(test_data['crimeaditionalinfo']), axis=-1)
test_data['predicted_labels'] = le.inverse_transform(test_predictions)
print(test_data[['crimeaditionalinfo', 'predicted_labels']])

976/976 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step
                                      crimeaditionalinfo  \
0      sir namaskar  mein ranjit kumar patrapaise neh...   
1              kotak mahindra bank fraud\r\nfraud amount   
2      the issue actually started when i got this ema...   
3      i am amit kumar from karwi chitrakoot i am tot...   
4      i have ordered  saree and  blouse from rinki s...   
...                                                  ...   
31224  a lady named rashmi probably a fake name had c...   
31225  i am mr chokhe ram  two pers mobile number wer...   
31226  mai bibekbraj maine pahle ki complain kar chuk...   
31227  received url link for updating kyc from mobile...   
31228  i saw add on facebook for job placement and i ...   

                                        predicted_labels  
0      Online and Social Media Related Crime-Cheating...  
1              Online Financial Fraud-UPI Related Frauds  
2      Cyber Attack/ Dependent Crimes-Tampering with ...  


In [12]:
output = test_data[['crimeaditionalinfo', 'predicted_labels']]

# Define the path and name for your output file
output_file_path = '/kaggle/working/predicted_data.csv'

# Save to CSV
output.to_csv(output_file_path, index=False)

# Display a message to confirm file has been saved
print(f"Predicted data saved to {output_file_path}")

Predicted data saved to /kaggle/working/predicted_data.csv
